In [1]:
import numpy as np #basic python library
import matplotlib.pyplot as plt #For plotting results
from matplotlib.pyplot import figure, legend, plot, show, title, xlabel, ylabel, ylim
import cv2 
import glob

## Import Landlab components
from landlab.components import DepressionFinderAndRouter #Pit filling
from landlab.components import PriorityFloodFlowRouter #Flow routing
from landlab.components import FlowAccumulator #FlowAccumulator 
from landlab.components import ChannelProfiler 
from landlab.components import SteepnessFinder
from landlab.components import ChiFinder
from landlab.components import ExponentialWeatherer
from landlab.components import DepthDependentDiffuser

#SPACE model
from landlab.components import Space #SPACE model
from landlab.components import SpaceLargeScaleEroder #basically SPACE 2.0 -- use this 
from landlab.components import FastscapeEroder #calculates the amount of erosion at each node
from landlab.components import SinkFillerBarnes #replaces pits with shallow gradients for draining

## Import Landlab utilities
from landlab import RasterModelGrid #Grid utility
from landlab import imshow_grid #For plotting results
from landlab.io import read_esri_ascii #to read in an ascii file
from landlab.io import read_asc_header #to read in the header of the ascii file
from landlab.io import write_esri_ascii 

%matplotlib inline

ModuleNotFoundError: No module named 'landlab.components.threshold_eroder.cfuncs'

In [ ]:
#Set grid parameters
num_rows = 100
num_columns = 100
node_spacing = 10.00
node_next_to_outlet = num_columns + 1
outlet_node = 0 

#Set Model Time Parameters
timestep = 100.0
run_time = 10000 #years
uplift_rate = .01 # m/year
uplift_per_timestep = uplift_rate * timestep

mg = RasterModelGrid((num_rows, num_columns), node_spacing)#Instantiate model grid

#Close all model boundary edges
mg.set_closed_boundaries_at_grid_edges(bottom_is_closed=True,left_is_closed=True,right_is_closed=True,top_is_closed=True)
mg.status_at_node[outlet_node] = mg.BC_NODE_IS_FIXED_VALUE

#Add field 'soil__depth' to the grid
mg.add_zeros('node', 'soil__depth')
mg.at_node['soil__depth'][:] = 2  #Set 2 m of initial soil depth at core nodes

np.random.seed(seed = 5000)#Set constant random seed for consistent topographic roughness

#Add field 'bedrock__elevation' to the grid
mg.add_zeros('bedrock__elevation', at='node')
mg.at_node["bedrock__elevation"][:] += (np.random.rand(len(mg.node_y)) / 100) #cm scale rn 
mg.at_node['bedrock__elevation'][outlet_node] = 0 # do this before the mg.at node stuff above
#Sum 'soil__depth' and 'bedrock__elevation' to yield 'topographic elevation' 
#when u change spacing high

#Add field ’topographic elevation’ to the grid
mg.add_zeros('node', 'topographic__elevation')
mg.at_node["topographic__elevation"][:] += mg.at_node["bedrock__elevation"]# + mg.at_node['soil__depth'][:]
mg.at_node["topographic__elevation"][:] += mg.at_node["soil__depth"]
#mg.at_node["topographic__elevation"][:] += mg.at_node["soil__depth"]

#outlet_node = mg.set_watershed_boundary_condition_outlet_id(0,mg['node']['topographic__elevation'], -9999.)
#mg.at_node['bedrock__elevation'][outlet_node] = 1 # do this before the mg.at node stuff above
#outlet_node = mg.set_watershed_boundary_condition_outlet_id(0,mg['node']['topographic__elevation'], -9999.)

mg.at_node['topographic__elevation'][outlet_node] = 0
mg.at_node['soil__depth'][outlet_node] = 0
print(outlet_node)


#Set grid parameters
num_rows = 100
num_columns = 100
node_spacing = 10.00
node_next_to_outlet = num_columns + 1
outlet_node = 0 

#Set Model Time Parameters
timestep = 100.0
run_time = 10000 #years
uplift_rate = .01 # m/year
uplift_per_timestep = uplift_rate * timestep

mg = RasterModelGrid((num_rows, num_columns), node_spacing)#Instantiate model grid

#Close all model boundary edges
mg.set_closed_boundaries_at_grid_edges(bottom_is_closed=True,left_is_closed=True,right_is_closed=True,top_is_closed=True)
mg.status_at_node[outlet_node] = mg.BC_NODE_IS_FIXED_VALUE

(mg,_) = read_esri_ascii(r"C:\Users\gsbir\Documents\EES_393\Export\sample_topo2_bedrock__elevation.txt", name = "bedrock__elevation", grid = mg)
(mg,_) = read_esri_ascii(r"C:\Users\gsbir\Documents\EES_393\Export\sample_topo2_topographic__elevation.txt", name = "topographic__elevation", grid = mg)
(mg,_) = read_esri_ascii(r"C:\Users\gsbir\Documents\EES_393\Export\sample_topo2_soil__depth.txt", name = "soil__depth", grid = mg)

In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)


In [ ]:
#Parameters for SPACE and Fastscape
K_sed = 0.0001
K_sp = 0.001
K_br = 0.0001
F_f = 0.0
phi = 0.0
H_star = 1.0
v_s = 5.0
m_sp = 0.5
n_sp = 1.0
sp_crit_sed = 0
sp_crit_br = 0

#Instantiate the Flow accumulator
fr = PriorityFloodFlowRouter(mg, flow_metric='D8', suppress_out = True)

#Instantiate the Flow accumulator
#fa = FlowAccumulator(mg, flow_director='D8') 

#Instantiate the depressionfinder and router as df
#df = DepressionFinderAndRouter(mg,pits = 'flow__sink_flag', reroute_flow = True) 

#instantiate the fascape eroder as fsc 
fsc = FastscapeEroder(mg, K_sp, m_sp, n_sp)

#Instantiate SPACE model with chosen parameters as sp
sp = SpaceLargeScaleEroder(mg, K_sed, K_br, F_f, phi, H_star, v_s, m_sp, n_sp, sp_crit_sed, sp_crit_br)# K_sed, K_br, F_f, phi, H_star, v_s, m_sp, n_sp, sp_crit_sed, sp_crit_br 

expweath = ExponentialWeatherer(mg)
DDdiff = DepthDependentDiffuser(mg)

expweath.calc_soil_prod_rate()
np.allclose(mg.at_node['soil_production__rate'][mg.core_nodes], 1.)


TRY RUNNING AT 10 SPACING FOR 100000 

In [ ]:

#run for 10 years
for x in range(5000): # at 10000 topo look doesn't change much
    mg.at_node['bedrock__elevation'][mg.core_nodes] += uplift_per_timestep  #adding uplift to the bedrock
    mg.at_node['topographic__elevation'][mg.core_nodes] = mg.at_node['bedrock__elevation'][mg.core_nodes] + mg.at_node['soil__depth'][mg.core_nodes]  #adding uplift to the bedrock
    #mg.at_node["topographic__elevation"][0] -= 0.001 # Uplift
    
    #fa.run_one_step()
    fr.run_one_step()        
    fsc.run_one_step(timestep)
    #DDdiff.run_one_step(1)
    print(x) 



In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)


saving grid to ascii


In [ ]:
grid_fields_to_save = ["topographic__elevation", "bedrock__elevation", "soil__depth"]

fname = r"C:\Users\gsbir\Documents\EES_393"  + "\\Export\\" + "sample_topo2.txt"

write_esri_ascii(fname, mg, grid_fields_to_save)

Running SPACE

In [ ]:

space_years = np.arange(1,1000,10000)

#run for 10 years
for x in range(1000): 
    mg.at_node['bedrock__elevation'][mg.core_nodes] += uplift_per_timestep  #adding uplift to the bedrock
    mg.at_node['topographic__elevation'][mg.core_nodes]+= uplift_per_timestep  #adding uplift to the bedrock
    #mg.at_node["topographic__elevation"][0] -= 0.001 # Uplift
    
    fr.run_one_step()
    #fa.run_one_step()
    #df.map_depressions()
    sp.run_one_step(dt = timestep)
    #DDdiff.run_one_step(1)
    print(x) 



In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)


In [ ]:
prf = ChannelProfiler(mg,
                      number_of_watersheds=1,
                      main_channel_only=True,
                      minimum_channel_threshold=node_spacing**2)

prf.run_one_step()

#Show the profile
plt.figure(1)
title_text = f"$K_{{sp}}$={.001}; $time$={1000000} yr; $dx$={1000000} m"
prf.plot_profiles(xlabel='distance upstream (m)', ylabel='elevation (m)', title=title_text)